# Criação de dim localizacao

##Imports

In [0]:
from pyspark.sql.functions import monotonically_increasing_id, col, when

##Selecionando dados da tabela vendas da camada prata

In [0]:
df_silver_localizacao = spark.sql("""
                      SELECT DISTINCT
                         uf_venda,
                         nome_uf,
                         municipio_venda,
                         latitude_municipio,
                         longitude_municipio,
                         capital,
                         latitude_uf,
                         longitude_uf,
                         regiao
                      FROM controlled_medication_data.silver_layer.vendas
                      """)

In [0]:
df_silver_localizacao.display()

In [0]:
df_silver_localizacao.filter(col("capital") == 1).display()

In [0]:
df_silver_localizacao.filter(col("uf_venda") == "DF").display()

## Tratando dados relacionados ao Distrito Federal

In [0]:
df_silver_localizacao =  df_silver_localizacao.fillna(-1, ['latitude_municipio', 'longitude_municipio'])

In [0]:
df_silver_localizacao = (
    df_silver_localizacao
    .withColumn(
        "latitude_uf",
        when(col("uf_venda") == "DF", 15.779029).otherwise(col("latitude_uf"))
    )
    .withColumn(
        "longitude_uf",
        when(col("uf_venda") == "DF", -47.929229).otherwise(col("longitude_uf"))
    )
    .withColumn(
        "capital",
        when((col("uf_venda") == "DF") & col("capital").isNull(), 0)
        .otherwise(col("capital"))
    )
    .withColumn(
        "regiao",
        when(col("uf_venda") == "DF", "Centro-Oeste").otherwise(col("regiao"))
    )
    .withColumn(
        "nome_uf",
        when(col("uf_venda") == "DF", "Distrito Federal").otherwise(col("nome_uf"))
    )
)

In [0]:
df_silver_localizacao.filter(col("uf_venda")=="DF").display()

## Criando um id para cada registro na dim (pk)

In [0]:
df_dim_localizacao = df_silver_localizacao.withColumn("id_localizacao", monotonically_increasing_id())

In [0]:
df_dim_localizacao.display()

In [0]:
df_dim_localizacao.write.mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("controlled_medication_data.gold_layer.dim_localizacao_venda")